In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow as tf
import json
import os
import sys
import cv2
import yaml
import numpy as np
from tensorflow.python.keras import backend as K
p = os.path.abspath('.')
sys.path.insert(1, p)
from models.lenet5Autoencode import LeNet
from data_import_and_preprocessing.dataset_formation import DataParser, ImageDataExtractor, LabelExtractor, \
    DataSetCreator
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2022-06-29 16:32:58.673899: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


"initialize Dataset and training params from params.yml"

In [2]:
print(os.getcwd())
os.chdir('/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition')
print(os.getcwd())
with open('params.yaml', 'r') as stream:
    params = yaml.safe_load(stream)
    no_epochs = params['training']['no_epochs']
    batch_size = params['training']['batch_size']
    loss = params['training']['loss']
    picture_width = params['preprocessing']['picture_width']
    picture_hight = params['preprocessing']['picture_hight']
    no_classes = params['preprocessing']['no_classes']

    data_dir = 'data/preprocessed'
    data_parser = DataParser(data_dir)
    image_data_extractor = ImageDataExtractor((picture_width, picture_hight, 1))
    label_extractor = LabelExtractor(no_classes=no_classes) #anzahl der classen
    dataset = DataSetCreator(data_parser, image_data_extractor, label_extractor, no_repeats=no_epochs)
    no_points = len(dataset)
    no_points_train = int(no_points * 0.8)
    no_points_val = int(no_points * 0.1)

    dataset_train = dataset.take(no_points=no_points_train)

/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition/inference
/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition


2022-06-29 16:33:04.330534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-29 16:33:04.936934: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:18:00.0, compute capability: 7.5


## load model

In [3]:
model_path = "/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition/training/model/trained_model.h5"
reconstructed_model = tf.keras.models.load_model(model_path)

2022-06-29 16:33:15.126145: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.10MiB (rounded to 6400000)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-06-29 16:33:15.126212: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-06-29 16:33:15.126237: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 12, Chunks in use: 12. 3.0KiB allocated for chunks. 3.0KiB in use in bin. 356B client-requested in use in bin.
2022-06-29 16:33:15.126256: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-06-29 16:33:15.126272: I tensorflow/core/common_run

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

## load img and infer class label

In [ ]:
path = '/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition/data/preprocessed/1100000001.png'
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
img_preprocessed  = img / 255

x = np.expand_dims(img_preprocessed, axis=0)


preds = reconstructed_model.predict(x)

## pretty the prediction for console output

In [ ]:
pred=np.around(preds,3)
inferedClass=""

prediction= pred[0]
inferedAccuracyFlooded=prediction[0]
inferedAccuracyLoaded=prediction[1]
inferedAccuracyDispersed=prediction[2]

classaccuracy = max(inferedAccuracyFlooded,inferedAccuracyDispersed,inferedAccuracyLoaded)
if classaccuracy == inferedAccuracyFlooded:
    inferedClass= "Flooded"
elif classaccuracy == inferedAccuracyLoaded:
    inferedClass= "Loaded"
elif classaccuracy == inferedAccuracyDispersed:
    inferedClass= "Despersed"
print(inferedClass+": "+str(classaccuracy))

## get dense autoencode auxilary putput

In [ ]:
reconstructed_model.layers[11].output
autoencodeLayer = K.function([reconstructed_model.layers[0].input], [reconstructed_model.layers[11].output])
autoencodeLayer_output = autoencodeLayer(x)[0]
autoencodeLayer_output

## asign Points

In [ ]:
from data_import_and_preprocessing.dataset_formation import DataParser, ImageDataExtractor, LabelExtractor, \
    DataSetCreator

print(os.getcwd())
os.chdir('/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition')
print(os.getcwd())
with open('params.yaml', 'r') as stream:
    params = yaml.safe_load(stream)
no_epochs = params['training']['no_epochs']
batch_size = params['training']['batch_size']
picture_width = params['preprocessing']['picture_width']
picture_hight = params['preprocessing']['picture_hight']
no_classes = params['preprocessing']['no_classes']

data_dir = 'data/preprocessed'
data_parser = DataParser(data_dir)
image_data_extractor = ImageDataExtractor((picture_width, picture_hight, 1))
label_extractor = LabelExtractor(no_classes=no_classes) #anzahl der classen
dataset = DataSetCreator(data_parser, image_data_extractor, label_extractor, no_repeats=no_epochs)
pathList = []
for imagePath in dataset.data_points:
    pathList.append(imagePath.path_to_data)

In [ ]:
from models.lenet5Autoencode import LeNet
import pandas as pd
#Create a clustering model
partialDataNo=100
points_n = len(pathList)/partialDataNo
clusters_n = 3
iteration_n = 100
i=0
model = LeNet

listOfAuxValues = []
valueLabelMapping = pd.DataFrame(columns=["point", "label"])


skippCounter=0
for path in pathList:
    skippCounter=skippCounter+1
    if  skippCounter%partialDataNo!=0:
        continue
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
    img_preprocessed  = img / 255
    x = np.expand_dims(img, axis=0)
    reconstructed_model.predict(x)
    autoencodeLayer = K.function([reconstructed_model.layers[0].input], [reconstructed_model.layers[11].output])
    autoencodeLayer_output = autoencodeLayer(x)[0]
    listOfAuxValues.append(autoencodeLayer_output)

    pred=np.around(preds,3)
    inferedClass=""

    prediction= pred[0]
    inferedAccuracyFlooded=prediction[0]
    inferedAccuracyLoaded=prediction[1]
    inferedAccuracyDispersed=prediction[2]

    classaccuracy = max(inferedAccuracyFlooded,inferedAccuracyDispersed,inferedAccuracyLoaded)
    if classaccuracy == inferedAccuracyFlooded:
        inferedClass= "Flooded"
    elif classaccuracy == inferedAccuracyLoaded:
        inferedClass= "Loaded"
    elif classaccuracy == inferedAccuracyDispersed:
        inferedClass= "Despersed"
    print(inferedClass+": "+str(classaccuracy))
    entry = pd.DataFrame.from_dict({
    "point": [autoencodeLayer_output],
    "label":  [inferedClass+": "+str(classaccuracy)]
    })
    valueLabelMapping = pd.concat([valueLabelMapping, entry], ignore_index=True)


## init clustering

In [ ]:
dataframe = []
for p in listOfAuxValues:
    print(p[0])
    dataframe.append(p[0])

points = tf.convert_to_tensor(dataframe)
points

In [ ]:
centroids = tf.Variable(tf.slice(tf.random.shuffle(points), [0, 0], [clusters_n, -1]))

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

distances = tf.reduce_sum(tf.square(tf.subtract(points_expanded, centroids_expanded)), 2)
assignments = tf.argmin(distances, 0)

means = []
for c in range(clusters_n):
    means.append(tf.reduce_mean(
      tf.gather(points,
                tf.reshape(
                  tf.where(
                    tf.equal(assignments, c)
                  ),[1,-1])
               ),[1]))

new_centroids = tf.concat(means, 0)

update_centroids =  tf.compat.v1.assign(centroids, new_centroids)
init = tf.compat.v1.global_variables_initializer()

In [ ]:
for step in range(iteration_n):
    [_, centroid_values, points_values, assignment_values] = [update_centroids, centroids, points, assignments]

    print("centroids", centroid_values)

plt.scatter(points_values[:, 0], points_values[:, 1], c=assignment_values, s=50, alpha=0.5)
plt.plot(centroid_values[:, 0], centroid_values[:, 1], 'kx', markersize=15)
plt.show()

In [ ]:
valueLabelMapping